# 제주 한달살기 숙소 추천을 위한 분석
### 대상 - 펜션, 민박
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- 호텔의 경우 가능 하겠지만 일반적인 한달살기 환경이 아니라는 가정

In [1]:
# !pip install selenium pandas bs4 xlrd requests tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd
import numpy as np

In [10]:
# 야놀자 펜션/풀빌라 - https://www.yanolja.com/sub-home/pension
# 제주 지역 - https://www.yanolja.com/pension/r-900593?advert=AREA&topAdvertiseMore=1

base_url = 'https://www.yanolja.com'
jeju_pension_url = '/pension/r-900593?advert=AREA&topAdvertiseMore=1'

driver = webdriver.Chrome()
driver.get(base_url + jeju_pension_url)
driver.implicitly_wait(10)

#### 실제에서는 스크롤수 확보...
while True:
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(1.0, 1.5), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(0.3, 1.2), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')

# 펜션리스트 확보

# 초이스랑 나머지로 구분됨
choice = soup.select('.PlaceListItemBanner_container__ARsIm')
other = soup.select('.PlaceListItemText_container__fUIgA')

yanolja_list = [choice, other]

pension_info1 = []
for type_list in yanolja_list:
    for pension in type_list:
        

        pension_name = pension.select_one('.PlaceListTitle_text__2511B').text.strip()

        try :
            pension_score = pension.select_one('.PlaceListScore_rating__3Glxf').text.strip()
            pension_reviewNum = int(pension.select_one('.PlaceListScore_reviewInfo__3QSCU').text.strip()[1:-1])
        except :
            pension_score = np.nan
            pension_reviewNum = np.nan

        try : 
            pension_price = int(pension.select_one('.PlacePriceInfoV2_discountPrice__1PuwK').text.strip().replace(',', ''))
        except :
            pension_price = np.nan

        try: 
            pension_url = pension.attrs['href']
        except:
            pension.select_one('a').attrs['href']

        pension_info1.append({'name' : pension_name,
                            'score' : pension_score,
                            'reviewNum' : pension_reviewNum,
                            'price' : pension_price,
                            'link' : pension_url})
        
pension_info1 = pd.DataFrame(pension_info1)
pension_info1.to_csv('./data/data1.csv', index=False)


In [12]:
pension_info1 = pd.read_csv('./data/data1.csv')
pension_info1

,name,score,reviewNum,price,link
0,제주 길리리조트,5.0,67.0,180000.0,/pension/1000100762
1,"제주 루마인펜션(비치썬라이즈,파티-퍼즈바 대여)",4.3,378.0,129000.0,/pension/1000100762
2,서귀포 선셋코스트 카라반&캠핑장,5.0,2.0,120000.0,/pension/1000100762
3,제주 선흘안풀빌라(넓은평수 감성숙소),4.9,15.0,240000.0,/pension/1000100762
4,서귀포 블루마운틴4255리조트,4.5,162.0,119000.0,/pension/1000100762
...,...,...,...,...,...
1670,제주 제주소랑앤팜스빌리지펜션,NaN,NaN,NaN,/pension/1000100762
1671,제주 다섯손가락펜션,NaN,NaN,90000.0,/pension/1000100762
1672,서귀포 해변의집영빈펜션(단독판매),NaN,NaN,100000.0,/pension/1000100762
1673,서귀포 행운클로버펜션,NaN,NaN,130000.0,/pension/1000100762


In [79]:
pension_info2 = []
driver = webdriver.Chrome()
for sub_url in pension_info1['link']:    
    driver.get(base_url + sub_url)
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    

    # 숙소 이름
    pension_name = driver.find_element(By.CSS_SELECTOR, '#__next > div > div > main > article > div.css-1cc3d9 > div.css-6pnu6y > div.css-11vo59c > h1').text.strip()

    # 주소
    pension_addr = driver.find_element(By.CLASS_NAME, 'address').text.strip()

    time.sleep(round(random.uniform(0.0, 2.0), 3))
    # 상세정보 클릭
    driver.find_element(By.CLASS_NAME, 'css-14fkj0f').click()
    driver.execute_script(f'window.scrollTo(0, 0)')
    time.sleep(round(random.uniform(0.0, 1.0), 3))

    # 상세정보 - 소개
    pension_intro = driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(1) > div > div.css-pw6xk > div').text.strip().replace('\n','')

    # 상세정보 - 편의시설
    facility = driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(2) > div > div.css-pw6xk')
    pension_text = facility.find_element(By.CLASS_NAME, 'css-7kp13n').text.strip().replace('\n','')
    pension_tag = facility.find_element(By.CLASS_NAME, 'css-bgisrg').text.strip()

    pension_info2.append({'name' : pension_name,
                          'addr' : pension_addr,
                          'intro' : pension_intro,
                          'tag' : pension_tag,
                          'facility' : pension_text})
    
    time.sleep(round(random.uniform(0.0, 3.0), 3))

pension_info2 = pd.DataFrame(pension_info2)
pension_info2.to_csv('./data/data2.csv', index=False)

    
    

,name,score,reviewNum,price,link
0,제주 길리리조트,5.0,67.0,180000.0,/pension/1000100762
1,"제주 루마인펜션(비치썬라이즈,파티-퍼즈바 대여)",4.3,378.0,129000.0,/pension/1000100762
2,서귀포 선셋코스트 카라반&캠핑장,5.0,2.0,120000.0,/pension/1000100762
3,제주 선흘안풀빌라(넓은평수 감성숙소),4.9,15.0,240000.0,/pension/1000100762
4,서귀포 블루마운틴4255리조트,4.5,162.0,119000.0,/pension/1000100762
...,...,...,...,...,...
1592,서귀포 농장&펜션설문대하우스,5.0,4.0,NaN,/pension/1000100762
1602,제주 스타하우스펜션,2.0,35.0,35100.0,/pension/1000100762
1612,제주 김녕몽생이펜션,4.5,2.0,188000.0,/pension/1000100762
1630,서귀포 데이스테이하우스펜션,3.0,2.0,180000.0,/pension/1000100762


In [154]:
temp = pension_info1[pension_info1['reviewNum'] > 0]
temp_stage2 = pd.DataFrame(columns = ['link', 'star', 'text'])

for sub_url in temp['link']:    
    pension_id = sub_url.split('/')[-1]
    review_url = f'https://place-site.yanolja.com/places/{pension_id}/review'
    driver = webdriver.Chrome()
    driver.get(review_url)

    #### 실제에서는 스크롤수 확보...
    for _ in range(3):
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(50, 100)})')
        time.sleep(round(random.uniform(1.0, 2.0), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
        time.sleep(round(random.uniform(0.5, 1.0), 3))     
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.5, 1.2), 3))

        # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

    review_soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_list = review_soup.select('.css-nkytaw')

    for review in review_list:
        review_text = review.select_one('.css-djrif0').text.strip()
        review_text = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", review_text)

        star_cnt = 0
        star_list = review.select_one('.css-1uvchqn').select('path')
        for star in star_list:
            star_cnt += star.get_attribute_list('fill').count('#fdbd00')

        temp_stage1 = []
        temp_stage1.append({'link' : sub_url,
                            'star' : star_cnt,
                            'text' : review_text})
        temp_stage1 = pd.DataFrame(temp_stage1)

        temp_stage2 = pd.concat([temp_stage2, temp_stage1])
    
    time.sleep(round(random.uniform(1.5, 3.0), 3))

temp_stage2 = pd.DataFrame(temp_stage2)
temp_stage2.to_csv('./data/review.csv', index=False)
temp_stage2

,link,star,text
0,/pension/10046493,5,창문 오션뷰는 쵝오 작은 주방에 있을건 다 있네요 푹신하고 수압 좋고 곽지 올레길도 굿굿
0,/pension/10046493,5,좋았습니다
0,/pension/10046493,5,너무 깨끗하고 멋진뷰에 감탄이 절로 나왔습니다
0,/pension/10046493,5,인테리어 이쁘고 테라스 개방감 있어서 좋았어요 세세한 부분에서 신경써주신게 보였고 ...
0,/pension/10046493,5,출장도 여행도 많이 다녔지만 여기 사장님의 친절과 서비스는 감동 그 자체입니다 와이...
...,...,...,...
0,/pension/1000109860,5,사장님도 친근하시고 주변 산책로도 너무 좋았습니다
0,/pension/1000109860,5,사장님 엄청 친절하시고 강아지 너무 귀엽고 애월 카페거리 엄청 가까워요 잘 놀다 ...
0,/pension/1000109860,5,너무 편하게 즐기다 갑니당
0,/pension/1000109860,5,신혼여행 중 애월근처로 급하게 잡은 숙소였는데 이런 좋은 곳에 자리가 남아 있었던 ...


In [152]:
len(temp_stage2)

55

In [ ]:
### 펜션리스트를 그대로 사용할 것인지?
### 네이버 한달살기 숙소 검색결과를 다시 크롤링 할것인지!
# 이를 통해서 이 숙소들의 위치와 정보를 확인할 것인가....